# Playground for downloading datasets

## Image net

In [1]:
import urllib
import requests
import json
import os
from glob import glob
import pandas as pd

In [4]:
def store_raw_images():
    url = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n04154340'
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    urls = response.read().decode('utf-8')
    return(urls)

## Plantnet / GBIF

In [2]:
base_url = "https://api.gbif.org/v1/"
plantnet_key = "7a3679ef-5582-4aaa-81f0-8c2545cafc81"

PN_data_path = "../datasets/plantnet_raw"
PN_data_file = "verbatim.txt"

In [3]:
!ls {PN_data_path}

citations.txt  meta.xml      multimedia.txt  rights.txt
dataset        metadata.xml  occurrence.txt  verbatim.txt


In [10]:
fp = "../datasets/plantnet_raw/occurrence.txt"
nrow = 5
df = pd.read_csv(fp, nrows=nrow, sep='\t')
nas = df.isna().sum()
print(nas[nas < nrow].index)
df['speciesKey']

Index(['gbifID', 'identifier', 'license', 'publisher', 'references',
       'datasetName', 'basisOfRecord', 'occurrenceID', 'individualCount',
       'occurrenceStatus', 'eventDate', 'year', 'month', 'day', 'countryCode',
       'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
       'acceptedNameUsageID', 'scientificName', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'specificEpithet', 'taxonRank',
       'taxonomicStatus', 'datasetKey', 'publishingCountry', 'lastInterpreted',
       'issue', 'mediaType', 'hasCoordinate', 'hasGeospatialIssues',
       'taxonKey', 'acceptedTaxonKey', 'kingdomKey', 'phylumKey', 'classKey',
       'orderKey', 'familyKey', 'genusKey', 'speciesKey', 'species',
       'genericName', 'acceptedScientificName', 'verbatimScientificName',
       'protocol', 'lastParsed', 'lastCrawled', 'repatriated', 'level0Gid',
       'level0Name', 'level1Gid', 'level1Name', 'level2Gid', 'level2Name',
       'level3Gid', 'level3Name',

0    2927069
1    5361909
2    2890212
3    2927303
4    8351737
Name: speciesKey, dtype: int64

In [12]:
df = pd.read_csv(fp, sep='\t')

In [17]:
test = df['countryCode'].value_counts()
test['US']

36256

In [19]:
test = df[~df['speciesKey'].isna() &
          (df['countryCode'] == 'US')]['speciesKey'].astype('int').value_counts()
test

3039194    706
3084015    679
3120060    360
2926557    311
5376075    310
          ... 
7819589      1
3170627      1
8184083      1
5384458      1
5414964      1
Name: speciesKey, Length: 2488, dtype: int64

In [4]:
!head "../datasets/plantnet_raw/occurrence.txt"

gbifID	abstract	accessRights	accrualMethod	accrualPeriodicity	accrualPolicy	alternative	audience	available	bibliographicCitation	conformsTo	contributor	coverage	created	creator	date	dateAccepted	dateCopyrighted	dateSubmitted	description	educationLevel	extent	format	hasFormat	hasPart	hasVersion	identifier	instructionalMethod	isFormatOf	isPartOf	isReferencedBy	isReplacedBy	isRequiredBy	isVersionOf	issued	language	license	mediator	medium	modified	provenance	publisher	references	relation	replaces	requires	rights	rightsHolder	source	spatial	subject	tableOfContents	temporal	title	type	valid	institutionID	collectionID	datasetID	institutionCode	collectionCode	datasetName	ownerInstitutionCode	basisOfRecord	informationWithheld	dataGeneralizations	dynamicProperties	occurrenceID	catalogNumber	recordNumber	recordedBy	individualCount	organismQuantity	organismQuantityType	sex	lifeStage	reproductiveCondition	behavior	establishmentMeans	occurrenceStatus	preparations	disposition	associatedReferences	ass

In [ ]:
pd.

In [15]:
verbatim_txt = os.path.join(PN_data_path, PN_data_file)

In [16]:
!head {verbatim_txt}

gbifID	abstract	accessRights	accrualMethod	accrualPeriodicity	accrualPolicy	alternative	audience	available	bibliographicCitation	conformsTo	contributor	coverage	created	creator	date	dateAccepted	dateCopyrighted	dateSubmitted	description	educationLevel	extent	format	hasFormat	hasPart	hasVersion	identifier	instructionalMethod	isFormatOf	isPartOf	isReferencedBy	isReplacedBy	isRequiredBy	isVersionOf	issued	language	license	mediator	medium	modified	provenance	publisher	references	relation	replaces	requires	rights	rightsHolder	source	spatial	subject	tableOfContents	temporal	title	type	valid	institutionID	collectionID	datasetID	institutionCode	collectionCode	datasetName	ownerInstitutionCode	basisOfRecord	informationWithheld	dataGeneralizations	dynamicProperties	occurrenceID	catalogNumber	recordNumber	recordedBy	individualCount	organismQuantity	organismQuantityType	sex	lifeStage	reproductiveCondition	behavior	establishmentMeans	occurrenceStatus	preparations	disposition	associatedMedia	associat

In [3]:
test = requests.get(base_url + "dataset/" + plantnet_key + "/metrics")

In [4]:
test_json = test.json()
test_json

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [41]:
!pip install gbif-dl

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.1 MB 1.9 MB/s            
     |████████████████████████████████| 62 kB 1.3 MB/s             
     |████████████████████████████████| 159 kB 22.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 53 kB 3.9 MB/s             
     |████████████████████████████████| 191 kB 13.0 MB/s            
     |████████████████████████████████| 270 kB 23.3 MB/s            
  Created wheel for gbif-dl: filename=gbif_dl-0.1.0-py3-none-any.whl size=16439 sha256=cc92c068150dfc09674db6823f95881172e4905b7c725ae7c91defb5cfea1c28
  Stored in directory: /root/.cache/pip/wheels/8c/71/af/e0782fd672b4e9e3d494e299d8c6e750f9ed4e3363e34a1d47
  Created wheel for aiostream: filename=aiostream-0.4.3-py3-none-any.whl

In [42]:
import gbif_dl

In [ ]:
queries = {
    "speciesKey": ['*'],
    "datasetKey": [
        "7a3679ef-5582-4aaa-81f0-8c2545cafc81",  # plantnet
    ],
    "limit": 100
}